# 第7回講義 宿題

## 課題. Tensorflowを用いて, MNISTを畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

**`tf` の以下のモジュールはこの回では使用できないように制限されています. 注意してください.**
```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [27]:
def homework(train_X, train_y, test_X):
    import time
    import matplotlib.pyplot as plt
    #%matplotlib inline
    rng = np.random.RandomState(1234)
    random_state = 42
    #Convolution layers [batchsize,picture_length,picture_width,color_channels]
    class Conv:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    class Conv_same:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='SAME'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    #Pooling layers [batchsize,picture_length,picture_width,color_channels]
    class Pooling:
        def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'):
            self.ksize = ksize
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    #Flatten layer    
    class Flatten:
        def f_prop(self, x):
            return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    #Full-link layers
    class Dense:
        def __init__(self, in_dim, out_dim, function=lambda x: x):
            # Xavier Initialization
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(in_dim + out_dim)),
                            high=np.sqrt(6/(in_dim + out_dim)),
                            size=(in_dim, out_dim)
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
            self.function = function

        def f_prop(self, x):
            return self.function(tf.matmul(x, self.W) + self.b)
    
    layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
        Conv_same((5, 5, 1, 20), tf.nn.relu),  # 28x28x 1 -> 28x28x20
        Pooling((1, 2, 2, 1)),            # 28x28x20 -> 14x14x20
        Conv((5, 5, 20, 50), tf.nn.relu), # 14x14x20 ->  10x 10x50
        Pooling((1, 2, 2, 1)),            #  10x 10x50 ->  5x 5x50
        Flatten(),
        Dense(5*5*50, 10, tf.nn.softmax)
    ]

    x = tf.placeholder(tf.float32, [None, 28, 28, 1])
    t = tf.placeholder(tf.float32, [None, 10])

    def f_props(layers, x):
        for layer in layers:
            x = layer.f_prop(x)
        return x

    y = f_props(layers, x)

    cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1)) # tf.log(0)によるnanを防ぐ
    train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    valid = tf.argmax(y, 1)
    
    n_epochs = 100
    batch_size = 100
    n_batches = train_X.shape[0]//batch_size

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    cost_plt =[]
    start_time=time.time()
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        cost_arry = []
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            _cost, _ = sess.run([cost,train], feed_dict={x: train_X[start:end], t: train_y[start:end]})
            cost_arry.append(_cost)
        cost_plt.append(np.sum(cost_arry))
        tttttt = time.time()-start_time
        print ("epoch",epoch,"time",tttttt,"cost",cost_plt[len(cost_plt)-1])

        pred_y = sess.run(valid, feed_dict={x: test_X})
        
    sess.close()
    plt.plot(np.log(cost_plt))
    
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

In [28]:
rng = np.random.RandomState(1234)
filter_shape=(5, 5, 1, 20)
fan_in = np.prod(filter_shape[:3])
fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
print(fan_in)
print(fan_out)
print(rng.uniform(low=-np.sqrt(6/(fan_in + fan_out)),high=np.sqrt(6/(fan_in + fan_out)),size=filter_shape).shape)
b=np.zeros((filter_shape[3]), dtype='float32')

25
500
(5, 5, 1, 20)


# Checker Cell (for student)

In [29]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import tensorflow as tf

del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.errors,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sdca,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test
]

def load_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    mnist_X = np.r_[mnist.train.images, mnist.test.images]
    mnist_y = np.r_[mnist.train.labels, mnist.test.labels]
    return train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))

    # validate for small dataset
    train_X_mini = train_X[:5000]
    train_y_mini = train_y[:5000]
    test_X_mini = test_X[:1000]
    test_y_mini = test_y[:1000]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
    
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

AttributeError: app

In [30]:
validate_homework()
#score_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 time 0.369398832321167 cost 112.248
epoch 1 time 0.7482202053070068 cost 102.073
epoch 2 time 1.2488698959350586 cost 76.6714
epoch 3 time 1.5868680477142334 cost 44.8425
epoch 4 time 1.971022367477417 cost 30.2629
epoch 5 time 2.3340132236480713 cost 24.2709
epoch 6 time 2.687563896179199 cost 20.8534
epoch 7 time 3.068906784057617 cost 18.7669
epoch 8 time 3.504521608352661 cost 17.2479
epoch 9 time 3.8531901836395264 cost 16.1904
epoch 10 time 4.189906120300293 cost 15.0544
epoch 11 time 4.636699199676514 cost 14.3714
epoch 12 time 4.97399377822876 cost 13.463
epoch 13 time 5.305962324142456 cost 12.8807
epoch 14 time 5.637435436248779 cost 12.3022
epoch 15 time 6.075819730758667 cost 11.8154
epoch 16 time 6.554885387420654 cost 11.3086
epoch 17 time 6.889982223510742 cost 10.9807
epo

In [31]:
score_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 time 4.082473516464233 cost 476.081
epoch 1 time 7.933734893798828 cost 142.761
epoch 2 time 10.940788269042969 cost 104.1
epoch 3 time 13.831298112869263 cost 83.3536
epoch 4 time 17.498560905456543 cost 70.0735
epoch 5 time 21.28737783432007 cost 60.6489
epoch 6 time 24.503108739852905 cost 54.085
epoch 7 time 28.631505250930786 cost 49.0585
epoch 8 time 32.54098176956177 cost 45.5697
epoch 9 time 36.73870849609375 cost 42.085
epoch 10 time 40.789835691452026 cost 39.3199
epoch 11 time 44.708088636398315 cost 37.0873
epoch 12 time 48.54865622520447 cost 35.2469
epoch 13 time 52.598029136657715 cost 33.2062
epoch 14 time 56.51204752922058 cost 32.0699
epoch 15 time 60.49246668815613 cost 30.7294
epoch 16 time 64.25076794624329 cost 29.363
epoch 17 time 67.96712875366211 cost 28.5394
epo